# Data Acquistion from Reddit

Go to <a href=#bookmark>bookmark</a>

### 2019-06-08 - Goal - Develop End-to-End Data Flow, at least at small scale.
# OR BUST

![](https://images.unsplash.com/photo-1515255384510-23e8b6a6ca3c?ixlib=rb-1.2.1&auto=format&fit=crop&w=1489&q=80)

---

## Libraries

In [1]:
# Install libs on this computer:
# !pip install praw
# !pip install pymongo
# !pip install psycopg2

In [2]:
import os             # file system stuff
import json           # digest json
import praw           # reddit API
import pandas as pd   # Dataframes
import pymongo        # MongoDB
import numpy as np    # math and arrays

from time import time # To time stuff

#DATA STORAGE
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

In [3]:
import helper     # Custom helper functions

---

## 1 Load Reddit keys

Step 3: Create your first Authorized Reddit Instance

In [5]:
# Define path to secret

secret_path = os.path.join(os.environ['HOME'], '.secret', 'reddit.json')
#secret_path = os.path.join(os.environ['HOME'], 'mia/.secret', 'reddit_api.json')

secret_path

'C:\\Users\\werlindo\\.secret\\reddit.json'

## 2 Load keys, Create Reddit Instance

In [6]:
keys = helper.get_keys(secret_path)

In [7]:
reddit = praw.Reddit(client_id=keys['client_id'] 
                     ,client_secret=keys['api_key']
                     ,username=keys['username']
                     ,password=keys['password']
                     ,user_agent='reddit_research accessAPI:v0.0.1 (by /u/FlatDubs)')

## 3 Obtain a Subreddit Instance(s) from your Reddit Instance

In [23]:
#politics = reddit.subreddit('politics')

In [24]:
got = reddit.subreddit('gameofthrones') #Let's start with got for now. If can dev flow for one, can just dupe for other

In [16]:
persons = """bran OR brandon stark 
                        OR jon snow OR jon 
                        OR khaleesi OR dany OR daenerys OR danyris
         """

In [48]:
persons = """
            "bran" OR 'brandon stark' OR 'jon snow' OR 'jon' 
                        OR 'khaleesi' OR 'dany' OR 'daenerys' OR 'danyris'
         """

In [49]:
persons = """
            "bran" OR "brandon stark" OR "jon snow" OR "jon" 
                        OR "khaleesi" OR "dany" OR "daenerys" OR "danyris"
         """

In [42]:
persons = """'or' OR 'brandon stark' 
                        or 'jon snow' or 'jon' 
                        or 'khaleesi' or 'dany' or 'daenerys' or 'danyris'
         """

In [46]:
persons = """(bran) OR (brandon stark) 
                        OR (jon snow) OR (jon) 
                        OR (khaleesi) OR (dany) OR (daenerys) OR (danyris)
         """

In [ ]:
version = 3

# Create search generator

# if version == 1:
#     got_search = got.search('bran' or 'brandon stark' 
#                             or 'jon snow' or 'jon' #will reddit authors be included in results?
#                             or 'khaleesi' or 'dany' or 'daenerys' or 'danyris', 
#                             sort='comments',
#                            limit= results_lim
#                            ,time_filter='month')
# elif version == 2:
#     got_search = got.search('doran' or 'davos', 
#                             sort='comments'
#                       #     limit= results_lim
#                            ,time_filter='week')
# else:
    # # # Create search generator

In [69]:
results_lim = 5

persons = "'doran' OR 'davos'"

got_search = got.search(persons, 
                        sort='comments',
                       limit= results_lim
                       ,time_filter='week')

# Compile submission into list
title = []
time_created = []
num_upvotes = []
num_comments = []
upvote_ratio = []
link_flair = []
redditor = []
sub_id = []
i=0

# for each submission     
for submission in got_search:
    i+=1
    title.append(submission.title)
    time_created.append(submission.created_utc)
    num_upvotes.append(submission.score)
    num_comments.append(submission.num_comments)
    upvote_ratio.append(submission.upvote_ratio)
    link_flair.append(submission.link_flair_text)
    redditor.append(submission.author)
    sub_id.append(submission.id)
#     body.append(?) #look at this later! is it comment[0]? 
    if i%5 == 0:
        print(f'{i} submissions completed')


df = pd.DataFrame(
    {'title': title,
     'time_created': time_created,
     'num_comments': num_comments,
     'num_upvotes': num_upvotes,
     'upvote_ratio': upvote_ratio,
     'link_flair': link_flair,
     'redditor': redditor,
     'id':sub_id
    })
df


,title,time_created,num_comments,num_upvotes,upvote_ratio,link_flair,redditor,id
0,[Spoilers] Winds of Winter was a perfect episode,1.559478e+09,158,222,0.74,Spoilers,FlayedMan345,bvwunn
1,[SPOILERS] The Great Council of 305 AC,1.559839e+09,48,19,0.65,Spoilers,MaegorBrightflame,bxir03
2,[No Spoilers] I found this horde of A Song of ...,1.559597e+09,2,4,0.62,No Spoilers,Skyking035,bwg9dd
3,[SPOILERS] Modified scenario 6 episodes of sea...,1.559681e+09,1,0,0.11,Spoilers,damgur,bwtqj1


In [75]:
persons = """
            "bran" OR 'brandon stark' OR 'jon snow' OR 'jon' 
                        OR 'khaleesi' OR 'dany' OR 'daenerys' OR 'danyris'
         """

In [76]:
results_lim = 1000

# persons = """"
# doran" OR "davos"
# """

got_search = got.search(persons, 
                        sort='comments',
                       limit= results_lim
                       ,time_filter='week')

sum(1 for s in got_search)



158